# Battle of Neighbors 

## Introduction and Data
Opening a New Shopping Mall in Bangalore, India

    Build a dataframe of neighborhoods in Bangalore, India by web scraping the data from Wikipedia page
    Get the geographical coordinates of the neighborhoods
    Obtain the venue data for the neighborhoods from Foursquare API
    Explore and cluster the neighborhoods
    Select the best cluster to open a new shopping mall


## Business Problem
This project is mainly focused on geospatial analysis of the Bangalore City to understand which would be the best place to open a new mall

### Import Libraries

In [ ]:
import numpy as np # library to handle data in a vectorized manner
!pip install geocoder
import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
!pip install folium
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

     |████████████████████████████████| 102kB 5.5MB/s 
Libraries imported.


### Scrap data from Wikipedia page into a DataFrame

In [ ]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore").text

In [ ]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [ ]:
# create a list to store neighborhood data
neighborhoodList = []

In [ ]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

neighborhoodList.remove("List of areas in Bangalore Cantonment")
neighborhoodList.remove("List of areas in Bengaluru Pete")
neighborhoodList.remove("List of neighbourhoods in Bangalore")

In [ ]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})
kl_df.head()

,Neighborhood
0,Adugodi
1,"Agara, Bangalore"
2,Ananthnagar
3,Anjanapura
4,Arekere


In [ ]:
# print the number of rows of the dataframe
kl_df.shape

(128, 1)

### Get the geographical coordinates

In [ ]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bangalore, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [ ]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [ ]:
coords

[[12.944020000000023, 77.60800000000006],
 [12.842830000000049, 77.48759000000007],
 [12.954080000000033, 77.54135000000008],
 [12.858110000000067, 77.55909000000008],
 [12.885680000000036, 77.59668000000005],
 [12.963480000000061, 77.61297000000008],
 [13.02753000000007, 77.65049000000005],
 [13.07728748449913, 77.65760577036913],
 [13.044710000000066, 77.55008000000004],
 [12.922310000000039, 77.56988000000007],
 [13.019643510687336, 77.65469211693214],
 [12.938980000000072, 77.57137000000006],
 [12.992220000000032, 77.53444000000007],
 [12.927350000000047, 77.67185000000006],
 [13.083010000000058, 77.54779000000008],
 [12.900090000000034, 77.60433000000006],
 [12.903080000000045, 77.62444000000005],
 [12.817530000000033, 77.67879000000005],
 [12.993330000000071, 77.66123000000005],
 [12.91488000000004, 77.61004000000008],
 [12.951940000000036, 77.54445000000004],
 [12.913110000000074, 77.71565000000004],
 [12.959700296061198, 77.57131179631511],
 [12.794010000000071, 77.700150000000

In [ ]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [ ]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [ ]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df.head(10)

(128, 3)


,Neighborhood,Latitude,Longitude
0,Adugodi,12.944020,77.608000
1,"Agara, Bangalore",12.842830,77.487590
2,Ananthnagar,12.954080,77.541350
3,Anjanapura,12.858110,77.559090
4,Arekere,12.885680,77.596680
5,Austin Town,12.963480,77.612970
6,Babusapalya,13.027530,77.650490
7,"Bagalur, Bangalore Urban",13.077287,77.657606
8,Bahubalinagar,13.044710,77.550080
9,Banashankari,12.922310,77.569880


In [ ]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

###  Create a map of Bangalore with neighborhoods superimposed on top

In [ ]:

address = 'Bangalore, Karnataka, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore, India 12.9791198, 77.5912997.


In [ ]:
# create map of Bangalore using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [ ]:
# save the map as HTML file
map_kl.save('map_kl.html')

### Use the Foursquare API to explore the neighborhoods 

In [ ]:
# define Foursquare Credentials and Version
# @hidden_cell
CLIENT_ID = '3Z3LMLWE5LMALWTAOHL50BJNY1W3QNRQTXCTCIYWEG0BDFXW' # Put Your Client Id
CLIENT_SECRET = 'IUYJOZKBMBLWE3MIKIMRZ2TZCC1V2EOQ5T4LDAPJIVSDCPLC' # Put You Client Secret 
VERSION = '20180604'

### Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [ ]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()['response']['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [ ]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6612, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adugodi,12.94402,77.608,Tommy Hilfiger,12.934552,77.611347,Clothing Store
1,Adugodi,12.94402,77.608,PVR IMAX,12.934595,77.611321,Movie Theater
2,Adugodi,12.94402,77.608,Truffles Ice & Spice,12.933443,77.614265,Burger Joint
3,Adugodi,12.94402,77.608,Lot Like Crêpes,12.936421,77.613284,Creperie
4,Adugodi,12.94402,77.608,Fenny's Lounge & Kitchen,12.935146,77.613536,Lounge


###  Let's check how many venues were returned for each neighorhood

In [ ]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adugodi,100,100,100,100,100,100
Ananthnagar,29,29,29,29,29,29
Anjanapura,4,4,4,4,4,4
Arekere,79,79,79,79,79,79
Austin Town,100,100,100,100,100,100
BTM Layout,89,89,89,89,89,89
Babusapalya,49,49,49,49,49,49
"Bagalur, Bangalore Urban",11,11,11,11,11,11
Bahubalinagar,19,19,19,19,19,19


### Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 235 uniques categories.


In [ ]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Clothing Store', 'Movie Theater', 'Burger Joint', 'Creperie',
       'Lounge', 'Indian Restaurant', 'Brewery', 'Dessert Shop',
       'Multiplex', 'Financial or Legal Service', 'Tea Room', 'Bar',
       'Café', 'Breakfast Spot', 'Mobile Phone Shop', 'Shopping Mall',
       'Gym', 'Donut Shop', 'Coffee Shop', 'Juice Bar', 'Ice Cream Shop',
       'Italian Restaurant', 'Chinese Restaurant', 'Bakery',
       'Indian Sweet Shop', "Men's Store", 'Yoga Studio', 'Smoke Shop',
       'Arcade', 'Gaming Cafe', 'Andhra Restaurant', 'Bookstore',
       'Kerala Restaurant', 'Fast Food Restaurant', 'Pizza Place',
       'Mexican Restaurant', 'Cosmetics Shop', 'Bowling Alley',
       'Tibetan Restaurant', 'Comfort Food Restaurant', 'Bistro',
       'Punjabi Restaurant', 'Eastern European Restaurant',
       'Thai Restaurant', 'Persian Restaurant',
       'Paper / Office Supplies Store', 'Seafood Restaurant',
       'Restaurant', 'Sandwich Place', 'Liquor Store'], dtype=object)

In [ ]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### Analyze Each Neighborhood

In [ ]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(6612, 236)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Outdoor Supply Store,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pool Hall,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Travel Agency,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Adugodi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adugodi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(127, 236)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Outdoor Supply Store,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pool Hall,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Travel Agency,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Adugodi,0.000000,0.00,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.0,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.030000,0.000000,0.000000,0.010000,0.010000,0.020000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.00,0.000000,0.000000,0.040000,0.000000,0.010000,0.000000,0.030000,0.01,0.000000,0.000000,0.010000,0.0000,0.010000,0.00,0.000000,0.000000,0.00,0.000000,0.050000,0.000000,0.000000,0.0000

In [ ]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

59

### Create a new DataFrame for Shopping Mall data only

In [ ]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [ ]:
kl_mall.head(10)

,Neighborhoods,Shopping Mall
0,Adugodi,0.010000
1,Ananthnagar,0.000000
2,Anjanapura,0.000000
3,Arekere,0.025316
4,Austin Town,0.010000
5,BTM Layout,0.011236
6,Babusapalya,0.000000
7,"Bagalur, Bangalore Urban",0.000000
8,Bahubalinagar,0.052632
9,Banashankari,0.010000


### Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Bangalore into 3 clusters.

In [ ]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 0, 1, 1, 1, 1, 0, 1], dtype=int32)

In [ ]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [ ]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Adugodi,0.010000,1
1,Ananthnagar,0.000000,1
2,Anjanapura,0.000000,1
3,Arekere,0.025316,0
4,Austin Town,0.010000,1


In [ ]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(127, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Adugodi,0.010000,1,12.94402,77.60800
1,Ananthnagar,0.000000,1,12.95408,77.54135
2,Anjanapura,0.000000,1,12.85811,77.55909
3,Arekere,0.025316,0,12.88568,77.59668
4,Austin Town,0.010000,1,12.96348,77.61297


In [ ]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(127, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
63,Kammanahalli,0.016667,0,13.014300,77.636850
26,Cooke Town,0.027778,0,13.003630,77.622830
27,"Cox Town, Bangalore",0.028986,0,12.995940,77.626440
104,"Shivajinagar, Bangalore",0.020000,0,12.987200,77.604010
98,Richmond Town,0.020000,0,12.962340,77.601230
39,Gowdanapalya,0.039216,0,12.909680,77.556750
89,Peenya,0.045455,0,13.031850,77.526790
45,Hoodi,0.020000,0,12.991980,77.715060
47,Hulimavu,0.017544,0,12.880640,77.601470
25,Chikkalasandra,0.029412,0,12.909950,77.550510


### Finally, let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### Examine Clusters
Cluster 0

In [ ]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
63,Kammanahalli,0.016667,0,13.014300,77.636850
26,Cooke Town,0.027778,0,13.003630,77.622830
27,"Cox Town, Bangalore",0.028986,0,12.995940,77.626440
104,"Shivajinagar, Bangalore",0.020000,0,12.987200,77.604010
98,Richmond Town,0.020000,0,12.962340,77.601230
39,Gowdanapalya,0.039216,0,12.909680,77.556750
89,Peenya,0.045455,0,13.031850,77.526790
45,Hoodi,0.020000,0,12.991980,77.715060
47,Hulimavu,0.017544,0,12.880640,77.601470
25,Chikkalasandra,0.029412,0,12.909950,77.550510


Cluster 1

In [ ]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
118,Vidyaranyapura,0.000000,1,13.079270,77.559140
80,Mathikere,0.000000,1,13.032350,77.558660
81,Milk Colony,0.000000,1,13.023254,77.579079
82,"Murphy Town, Bangalore",0.011765,1,12.980030,77.627760
74,Madiwala,0.010000,1,12.920520,77.620900
84,Nagarbhavi,0.000000,1,12.956240,77.509360
85,Nandini Layout,0.012346,1,13.014810,77.538910
86,Nayandahalli,0.000000,1,12.942050,77.521000
119,"Vijayanagar, Bangalore",0.000000,1,13.076000,77.652380
123,Wilson Garden,0.000000,1,12.949690,77.596790


Cluster 2

In [ ]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
53,Jalahalli,0.125000,2,13.05450,77.52658
105,"Siddapura, Bangalore",0.067797,2,12.95613,77.73196
28,Dasarahalli,0.111111,2,13.04224,77.51507
94,Ramachandrapura,0.066667,2,13.06124,77.55112


### Observation
Most of the shopping malls are concentrated in the central area of the Bangalore city, with the highest number in cluster 1 and moderate number in cluster 0. On the other hand, cluster 2 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 1 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 2 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 1 which already have high concentration of shopping malls and suffering from intense competition.